<div align='center'><font size="6" color="#F39C12">Getting started with Natural Language Processing</font></div>
<div align='center'><font size="5" color="#F39C12">A general Introduction</font></div>
<hr>


<p style='text-align:justify'><b>Key Objectives:</b>This notebook comes as a first part to the **Getting started with NLP Notebooks** that I am writing. This notebook explains the concepts of NLP with respect to this current competition. NLP is the field of study that focuses on the interactions between human language and computers. NLP sits at the intersection of computer science, artificial intelligence, and computational linguistics[[source](https://en.wikipedia.org/wiki/Natural_language_processing)]. NLP is a way for computers to analyze, understand, and derive meaning from human language in a smart and useful way.</p>

<b>Notebooks in this series</b>

  <ul>
      <li><a href="https://www.kaggle.com/parulpandey/getting-started-with-nlp" target="_blank">Part 1: Getting started with NLP : A General Introduction </a></li>
      <li><a href="https://www.kaggle.com/parulpandey/getting-started-with-nlp-2-countvectorizer" target="_blank">Part 2: Getting started with NLP(2)- CountVectorizer </a></li></ul>

<hr>

## Dataset
The datasets contains a set of tweets which have been divided into a training and a test set. The training set contains a target column identifying whether the tweet pertains to a real disasteror not.

Our job is to create a ML model to predict whether the test set tweets belong to a disaster or not, in the form of 1 or 0.This is a classic case of a Binary Classification problem. 

## Understanding the Evaluation Metric

Evaluation metrics are used to measure the quality of the statistical or machine learning model.There are many different types of evaluation metrics available to test a model. These include classification accuracy, logarithmic loss etc. For this particluar problem, our submissions will be evaluated using **F1** between the predicted and expected answers. 

The **F score**, also called the **F1 score** or **F measure**, is a measure of a test’s accuracy. 

The F score is defined as the weighted harmonic mean of the test’s precision and recall. 

![](https://imgur.com/nC4QwrO.png)

- Precision, also called the positive predictive value, is the proportion of positive results that truly are positive. 

- Recall, also called sensitivity, is the ability of a test to correctly identify positive results to get the true positive rate. 

![](https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/350px-Precisionrecall.svg.png)
*source:https://en.wikipedia.org/wiki/Precision_and_recall*

The  F score reaches the best value, meaning perfect precision and recall, at a value of 1. The worst F score, which means lowest precision and lowest recall, would be a value of 0. 



### Why is this Useful?
The F score is used to measure a test’s accuracy, and it balances the use of precision and recall to do it. The F score can provide a more realistic measure of a test’s performance by using both precision and recall. The F score is often used in information retrieval for measuring search, document classification, and query classification performance. 
          
          
source: https://deepai.org/machine-learning-glossary-and-terms/f-score          

# Table of Contents
* [1. Importing the necessary libraries](#imports)
- [2. Reading the datasets](#reading)
- [3. Basic EDA](#eda)
- [4. Text data processing](#processing)
- [5. Transforming tokens to vectors](#vectorization)
- [6. Buiding a Text Classification model](#model)

# <a name="imports"></a>1. Importing the necessary libraries

In [1]:
import numpy as np 
import pandas as pd 

# text processing libraries
import re
import string
import nltk
from nltk.corpus import stopwords

# XGBoost
import xgboost as xgb
from xgboost import XGBClassifier
# sklearn 
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV,StratifiedKFold,RandomizedSearchCV

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'xgboost'

# <a name="reading"></a> 2. Reading the datasets

In [ ]:
# List files available
print(os.listdir("../input/"))

In [ ]:
#Training data
train = pd.read_csv('../input/train.csv')
print('Training data shape: ', train.shape)
train.head()

The training data has 7613 observations and 5 features including the TARGET (the label we want to predict).

In [ ]:
# Testing data 
test = pd.read_csv('../input/test.csv')
print('Testing data shape: ', test.shape)
test.head()

Note the test data doesn't have the target column

# <a name="eda"></a> 3. Basic EDA

## Missing values

In [ ]:
#Missing values in training set
train.isnull().sum()

The columns denote the following: 

- The `text` of a tweet
- A `keyword` from that tweet
- The `location` the tweet was sent from

In [ ]:
#Missing values in test set
test.isnull().sum()

A lot of values are missing in the `location` column in both the training and the testing set.

## Exploring the Target Column

* ** Distribution of the Target Column**

We have to predict whether a given tweet is about a real disaster or not. - If so, predict a 1. If not, predict a 0.

In [ ]:
train['target'].value_counts()

In [ ]:
sns.barplot(train['target'].value_counts().index,train['target'].value_counts(),palette='rocket')

* **Exploring the Target Column**
Let's look at what the disaster and the non disaster tweets look like

In [ ]:
# A disaster tweet
disaster_tweets = train[train['target']==1]['text']
disaster_tweets.values[1]

In [ ]:
#not a disaster tweet
non_disaster_tweets = train[train['target']==0]['text']
non_disaster_tweets.values[1]

## Exploring the 'keyword' column
The keyword column denotes a keyword from the tweet.Let's look at the top 20 keywords in the training data

In [ ]:
sns.barplot(y=train['keyword'].value_counts()[:20].index,x=train['keyword'].value_counts()[:20],
            orient='h')

Let's see how often the word 'disaster' come in the dataset and whether this help us in determining whether a tweet belongs to a disaster category or not.

In [ ]:
train.loc[train['text'].str.contains('disaster', na=False, case=False)].target.value_counts()

## Exploring the 'location' column
Even though the column `location` has a number of missing values, let's see the top 20 locations present in the dataset. Since some of the locations are repeated, this will require some bit of cleaning.

In [ ]:
# Replacing the ambigious locations name with Standard names
train['location'].replace({'United States':'USA',
                           'New York':'USA',
                            "London":'UK',
                            "Los Angeles, CA":'USA',
                            "Washington, D.C.":'USA',
                            "California":'USA',
                             "Chicago, IL":'USA',
                             "Chicago":'USA',
                            "New York, NY":'USA',
                            "California, USA":'USA',
                            "FLorida":'USA',
                            "Nigeria":'Africa',
                            "Kenya":'Africa',
                            "Everywhere":'Worldwide',
                            "San Francisco":'USA',
                            "Florida":'USA',
                            "United Kingdom":'UK',
                            "Los Angeles":'USA',
                            "Toronto":'Canada',
                            "San Francisco, CA":'USA',
                            "NYC":'USA',
                            "Seattle":'USA',
                            "Earth":'Worldwide',
                            "Ireland":'UK',
                            "London, England":'UK',
                            "New York City":'USA',
                            "Texas":'USA',
                            "London, UK":'UK',
                            "Atlanta, GA":'USA',
                            "Mumbai":"India"},inplace=True)

sns.barplot(y=train['location'].value_counts()[:5].index,x=train['location'].value_counts()[:5],
            orient='h')

# <a name="processing"></a> 4. Text Data Preprocessing

## 1. Data Cleaning

Before we start with any NLP project we need to pre-process the data to get it all in a consistent format.We need to clean, tokenize and convert our data into a matrix. Some of the  basic text pre-processing techniques includes:

* Make text all **lower case** or **uppercase** so that the algorithm does not treat the same words in different cases as different
* **Removing Noise** i.e everything that isn’t in a standard number or letter i.e Punctuation, Numerical values,  common non-sensical text (/n)
* **Tokenization**: Tokenization is just the term used to describe the process of converting the normal text strings into a list of tokens i.e words that we actually want. Sentence tokenizer can be used to find the list of sentences and Word tokenizer can be used to find the list of words in strings.
* **Stopword Removal**: Sometimes, some extremely common words which would appear to be of little value in helping select documents matching a user need are excluded from the vocabulary entirely. These words are called stop words

### More data cleaning steps after tokenization:

* **Stemming**: Stemming is the process of reducing inflected (or sometimes derived) words to their stem, base or root form — generally a written word form. Example if we were to stem the following words: “Stems”, “Stemming”, “Stemmed”, “and Stemtization”, the result would be a single word “stem”.
* **Lemmatization**: A slight variant of stemming is lemmatization. The major difference between these is, that, stemming can often create non-existent words, whereas lemmas are actual words. So, your root stem, meaning the word you end up with, is not something you can just look up in a dictionary, but you can look up a lemma. Examples of Lemmatization are that “run” is a base form for words like “running” or “ran” or that the word “better” and “good” are in the same lemma so they are considered the same.
* Parts of speech tagging
* Create bi-grams or tri-grams
And more...

However, it is not necessary that you would need to use all these steps. The usage depends on your problem at hand. Sometimes removal of stop words helps while at other times, this might not help.Here is a nice table taken from the blog titled : [All you need to know about Text Preprocessing for Machine Learning & NLP](https://kavita-ganesan.com/text-preprocessing-tutorial/#.Xi2BhhczZTY) that summarizes how much preprocessing you should be performing on your text data:

![](https://kavita-ganesan.com/wp-content/uploads/2019/02/Screen-Shot-2019-02-23-at-1.36.52-PM-590x270.png)

In [ ]:
# A quick glance over the existing data
train['text'][:5]

In [ ]:
# Applying a first round of text cleaning techniques

def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

# Applying the cleaning function to both test and training datasets
train['text'] = train['text'].apply(lambda x: clean_text(x))
test['text'] = test['text'].apply(lambda x: clean_text(x))

# Let's take a look at the updated text
train['text'].head()

Just for fun let's create a wordcloud of the clean text to see the most dominating words in the tweets.

In [ ]:
from wordcloud import WordCloud
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[26, 8])
wordcloud1 = WordCloud( background_color='white',
                        width=600,
                        height=400).generate(" ".join(disaster_tweets))
ax1.imshow(wordcloud1)
ax1.axis('off')
ax1.set_title('Disaster Tweets',fontsize=40);

wordcloud2 = WordCloud( background_color='white',
                        width=600,
                        height=400).generate(" ".join(non_disaster_tweets))
ax2.imshow(wordcloud2)
ax2.axis('off')
ax2.set_title('Non Disaster Tweets',fontsize=40);

## 2. Tokenization

Tokenization is a process that splits an input sequence into so-called tokens where the tokens can be a word, sentence, paragraph etc. Base upon the type of tokens we want, tokenization can be of various types, for instance

In [6]:
text = "you coming , aren't you"
tokenizer1 = nltk.tokenize.WhitespaceTokenizer()
tokenizer2 = nltk.tokenize.TweetTokenizer()
tokenizer3 = nltk.tokenize.WordPunctTokenizer()
tokenizer4 = nltk.tokenize.RegexpTokenizer(r'\w+')

print("Example Text: ",text)
print("------------------------------------------------------------------------------------------------")
print("Tokenization by whitespace:- ",tokenizer1.tokenize(text))
print("Tokenization by words using Treebank Word Tokenizer:- ",tokenizer2.tokenize(text))
print("Tokenization by punctuation:- ",tokenizer3.tokenize(text))
print("Tokenization by regular expression:- ",tokenizer4.tokenize(text))

Example Text:  you coming , aren't you
------------------------------------------------------------------------------------------------
Tokenization by whitespace:-  ['you', 'coming', ',', "aren't", 'you']
Tokenization by words using Treebank Word Tokenizer:-  ['you', 'coming', ',', "aren't", 'you']
Tokenization by punctuation:-  ['you', 'coming', ',', 'aren', "'", 't', 'you']
Tokenization by regular expression:-  ['you', 'coming', 'aren', 't', 'you']


In [ ]:
# Tokenizing the training and the test set
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
train['text'] = train['text'].apply(lambda x: tokenizer.tokenize(x))
test['text'] = test['text'].apply(lambda x: tokenizer.tokenize(x))
train['text'].head()

## 3. Stopwords Removal

Now, let's get rid of the stopwords i.e words which occur very frequently but have no possible value like **a, an, the, are **etc.

In [ ]:
def remove_stopwords(text):
    """
    Removing stopwords belonging to english language
    
    """
    words = [w for w in text if w not in stopwords.words('english')]
    return words


train['text'] = train['text'].apply(lambda x : remove_stopwords(x))
test['text'] = test['text'].apply(lambda x : remove_stopwords(x))
train.head()

## 4. Token normalization

Token normalisation means converting different tokens to their base forms. This can be done either by:

- **Stemming** :  removing and replacing suffixes to get to the root form of the word, which is called the **stem** for instance cats - cat, wolves - wolv 
- **Lemmatization** : Returns the base or dictionary form of a word, which is known as the **lemma** 

[*source*](https://www.coursera.org/learn/language-processing/lecture/SCd4G/text-preprocessing)

In [ ]:
# Stemming and Lemmatization examples
text = "feet cats wolves talked"

tokenizer = nltk.tokenize.TreebankWordTokenizer()
tokens = tokenizer.tokenize(text)

# Stemmer
stemmer = nltk.stem.PorterStemmer()
print("Stemming the sentence: ", " ".join(stemmer.stem(token) for token in tokens))

# Lemmatizer
lemmatizer=nltk.stem.WordNetLemmatizer()
print("Lemmatizing the sentence: ", " ".join(lemmatizer.lemmatize(token) for token in tokens))

It is important to note here that stemming and lemmatization sometimes donot necessarily improve results as at times we donot want to trim words but rather preserve their original form. Hence their usage actually differs from problem to problem. For this problem, I will not use these techniques.

In [ ]:
# After preprocessing, the text format
def combine_text(list_of_text):
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ' '.join(list_of_text)
    return combined_text

train['text'] = train['text'].apply(lambda x : combine_text(x))
test['text'] = test['text'].apply(lambda x : combine_text(x))
train['text']
train.head()

### Getting it all together- A Text Preprocessing Function
This concludes the pre-processing part. It will be prudent to convert all the steps undertaken into a function for better reusability.

In [ ]:
# text preprocessing function
def text_preprocessing(text):
    """
    Cleaning and parsing the text.

    """
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    
    nopunc = clean_text(text)
    tokenized_text = tokenizer.tokenize(nopunc)
    remove_stopwords = [w for w in tokenized_text if w not in stopwords.words('english')]
    combined_text = ' '.join(remove_stopwords)
    return combined_text

# <a name="vectorization"></a>  5. Transforming tokens to a vector
After the initial preprocessing phase, we need to transform text into a meaningful vector (or array) of numbers. This can be done by a number of tecniques:

## Bag of Words

The bag-of-words is a representation of text that describes the occurrence of words within a document. It involves two things:

- A vocabulary of known words.
- A measure of the presence of known words.

Why is it is called a “bag” of words? That is because any information about the order or structure of words in the document is discarded and the model is only concerned with whether the known words occur in the document, not where they occur in the document.

For example, 

![](https://imgur.com/jWqtRP1.png)

*source:[Natural Language Processing course on coursera](https://www.coursera.org/learn/language-processing/home/welcome)*

We can do this using scikit-learn's CountVectorizer, where every row will represent a different tweet and every column will represent a different word.

### Bag of Words - Countvectorizer Features

[Countvectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) converts a collection of text documents to a matrix of token counts. It is important to note here that CountVectorizer comes with a lot of options to automatically do preprocessing, tokenization, and stop word removal.However, i did all the process manually above to just get a better understanding. Let's use a vanilla implementation of the countvectorizer without specifying any parameters.


In [ ]:
count_vectorizer = CountVectorizer()
train_vectors = count_vectorizer.fit_transform(train['text'])
test_vectors = count_vectorizer.transform(test["text"])

## Keeping only non-zero elements to preserve space 
print(train_vectors[0].todense().shape)
print(len(count_vectorizer.get_feature_names()))

### TFIDF Features

A problem with the Bag of Words approach is that highly frequent words start to dominate in the document (e.g. larger score), but may not contain as much “informational content”. Also, it will give more weight to longer documents than shorter documents.

One approach is to rescale the frequency of words by how often they appear in all documents so that the scores for frequent words like “the” that are also frequent across all documents are penalized. This approach to scoring is called Term Frequency-Inverse Document Frequency, or TF-IDF for short, where:

**Term Frequency: is a scoring of the frequency of the word in the current document.**

```
TF = (Number of times term t appears in a document)/(Number of terms in the document)
```

**Inverse Document Frequency: is a scoring of how rare the word is across documents.**

```
IDF = 1+log(N/n), where, N is the number of documents and n is the number of documents a term t has appeared in.
```

In [ ]:
tfidf = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1, 2))
train_tfidf = tfidf.fit_transform(train['text'])
test_tfidf = tfidf.transform(test["text"])


# <a name="model"></a> 6. Building a Text Classification model
Now the data is ready to be fed into a classification model. Let's create a basic claasification model using commonly used classification algorithms and see how our model performs.

## Logistic Regression Classifier

In [ ]:
# Fitting a simple Logistic Regression on Counts
clf = LogisticRegression(C=1.0)
scores = model_selection.cross_val_score(clf, train_vectors, train["target"], cv=5, scoring="f1")
scores

In [ ]:
clf.fit(train_vectors, train["target"])

In [ ]:
# Fitting a simple Logistic Regression on TFIDF
clf_tfidf = LogisticRegression(C=1.0)
scores = model_selection.cross_val_score(clf_tfidf, train_tfidf, train["target"], cv=5, scoring="f1")
scores

It appears the countvectorizer gives a better performance than TFIDF in this case.

## Naives Bayes Classifier
Well, this is a decent score. Let's try with another model that is said to work well with text data : Naive Bayes.

In [ ]:
# Fitting a simple Naive Bayes on Counts
clf_NB = MultinomialNB()
scores = model_selection.cross_val_score(clf_NB, train_vectors, train["target"], cv=5, scoring="f1")
scores

In [ ]:
clf_NB.fit(train_vectors, train["target"])

In [ ]:
# Fitting a simple Naive Bayes on TFIDF
clf_NB_TFIDF = MultinomialNB()
scores = model_selection.cross_val_score(clf_NB_TFIDF, train_tfidf, train["target"], cv=5, scoring="f1")
scores

well the naive bayes on TFIDF features scores much better than logistic regression model. 

In [ ]:
clf_NB_TFIDF.fit(train_tfidf, train["target"])

## XGBoost

In [ ]:
import xgboost as xgb
clf_xgb = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
scores = model_selection.cross_val_score(clf_xgb, train_vectors, train["target"], cv=5, scoring="f1")
scores

In [ ]:
import xgboost as xgb
clf_xgb_TFIDF = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
scores = model_selection.cross_val_score(clf_xgb_TFIDF, train_tfidf, train["target"], cv=5, scoring="f1")
scores



## Making the submission

In [ ]:
def submission(submission_file_path,model,test_vectors):
    sample_submission = pd.read_csv(submission_file_path)
    sample_submission["target"] = model.predict(test_vectors)
    sample_submission.to_csv("submission.csv", index=False)

In [ ]:
submission_file_path = "../input/sample_submission.csv"
test_vectors=test_tfidf
submission(submission_file_path,clf_NB_TFIDF,test_vectors)